In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


data = pd.read_csv("Admission_Predict.csv")
data.head()

,Sno,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceOfAdmit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [15]:
data = data.drop(['Sno'], axis=1)
data.head()

,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceOfAdmit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [18]:
X=data.drop('ChanceOfAdmit', axis=1)
y=data['ChanceOfAdmit']
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=42)

###Scale down featuResourceWarning
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
X_train

array([[ 0.45711129,  0.42466178, -0.057308  , ..., -1.05965163,
         0.13986648,  0.92761259],
       [ 0.1022887 ,  0.42466178, -0.057308  , ..., -0.50194025,
         0.36110014, -1.07803625],
       [ 2.05381293,  2.08593034,  1.6892215 , ...,  1.17119391,
         2.25009529,  0.92761259],
       ...,
       [-0.96217907, -0.40597251, -0.93057275, ..., -0.50194025,
        -0.62594237,  0.92761259],
       [-1.31700165, -1.40273364, -1.8038375 , ..., -1.61736302,
        -2.27668588, -1.07803625],
       [-0.25253389, -0.23984565, -0.93057275, ...,  0.05577114,
        -0.57488845, -1.07803625]])

In [20]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

2025-10-30 16:39:01.746372: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-30 16:39:01.794356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-30 16:39:01.794398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-30 16:39:01.795780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-30 16:39:01.803020: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-30 16:39:01.803471: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [22]:
model=Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)), ###first hidden layer connected with input layer
    Dense(32,activation='relu'), ###HL2
    Dense(1,activation='sigmoid')
])

In [23]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2625 (10.25 KB)
Trainable params: 2625 (10.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [27]:
tensorflow_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)


In [28]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
history= model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
10/10 [==============================] - 1s 42ms/step - loss: 0.5912 - accuracy: 0.0000e+00 - val_loss: 0.5456 - val_accuracy: 0.0000e+00
Epoch 2/100
10/10 [==============================] - 0s 13ms/step - loss: 0.5565 - accuracy: 0.0000e+00 - val_loss: 0.5426 - val_accuracy: 0.0000e+00
Epoch 3/100
10/10 [==============================] - 0s 19ms/step - loss: 0.5515 - accuracy: 0.0000e+00 - val_loss: 0.5369 - val_accuracy: 0.0000e+00
Epoch 4/100
10/10 [==============================] - 0s 14ms/step - loss: 0.5483 - accuracy: 0.0000e+00 - val_loss: 0.5379 - val_accuracy: 0.0000e+00
Epoch 5/100
10/10 [==============================] - 0s 12ms/step - loss: 0.5475 - accuracy: 0.0000e+00 - val_loss: 0.5357 - val_accuracy: 0.0000e+00
Epoch 6/100
10/10 [==============================] - 0s 16ms/step - loss: 0.5469 - accuracy: 0.0000e+00 - val_loss: 0.5387 - val_accuracy: 0.0000e+00
Epoch 7/100
10/10 [==============================] - 0s 12ms/step - loss: 0.5468 - accuracy: 0.0000e

In [30]:
model.save('model.h5')


/home/parul/pythonAi/collegeANNApp/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
%load_ext tensorboard

In [33]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 308838), started 0:00:13 ago. (Use '!kill 308838' to kill it.)